In [ ]:
# Install required packages
!pip install PyAthena[SQLAlchemy]
!pip install CurrencyConverter

# Import necessary libraries
import numpy as np
import pandas as pd
import json
import datetime
from datetime import timedelta
from sqlalchemy import create_engine
from currency_converter import CurrencyConverter, ECB_URL
import tr_dash_util as util
from tr_dash_util import clean_eReg_dataframe

# Define constants and configurations
S3_STAGING_DIR = "s3://ets-aws-plalab-dii-prod-analyticsbucket-1ktrlhzbrcbkb/athena_query_results/"
ATHENA_REGION = "us-east-1"
APPLICATION_ID = '01845c7c-fa6d-4788-9ddd-cd888c977f36'
ENV = 'prod'
USER_TYPE = 'external_user'
EVENT_NAMES = ['ProfileCreated', 'OrderSubmitted']
START_DATE = datetime.datetime(2024, 2, 1)
END_DATE = datetime.datetime(2024, 2, 10)
EXTENDED_END_DATE = datetime.datetime(2024, 5, 27)  # New end date

# SKU lists
NON_PREP_SKUS = [
    '-2002', '-2001', '-1111', '4001', '4002', '4003', '4004', '4005', '4006', '4007', '4008', '4009', '4010', '4011', 
    '4012', '4013', '4014', '4015', '4016', '4017', '4018', '4019', '4020', '4028', '4034', '4035', '4091', '4092', '4093', 
    '4321', '5000', '5001', '5002', '5003', '5004', '5005', '5006', '5007', '5009', '5010', '5020', '5021', '5022', '5023', 
    '5024', '5025', '5026', '5027', '5028', '5029', '5030', '5031', '5032', '5033', '5035', '5037', '5040', '5041', '5042', 
    '5043', '5044', '5045', '5046', '5047', '5048', '5049', '5050', '5051', '5052', '5053', '5054', '5055', '5056', '5057', 
    '5058', '5059', '5060', '5061', '5062', '5063', '5064', '5065', '5066', '5067', '5068', '5069', '5070', '5071', '5072', 
    '5073', '5074', '5075', '5076', '5077', '5078', '5079', '5080', '5081', '5082', '5083', '5084', '5085', '5086', '5087', 
    '5088', '5089', '5091', '5093', '5094', '5095', '5096', '5097', '5098', '5099', '5100', '5101', '5102', '5103', '5104', 
    '5105', '5106', '5107', '5108', '5109', '5110', '5111', '5112', '5113', '5114', '5115', '5116', '5117', '5118', '5119', 
    '5120', '5121', '5122', '5123', '5124', '5125', '5126', '5127', '5128', '5129', '5130', '5131', '5132', '5133', '5134', 
    '5135', '5136', '5137', '5138', '5139', '5140', '5142', '5143', '5144', '5145', '5146', '5147', '5148', '5149', '5150', 
    '5151', '5152', '5154', '5155', '5156', '5157', '5159', '5160', '5161', '5162', '5163', '5164', '5165', '5166', '5167', 
    '5168', '5169', '5170', '5171', '5172', '5173', '5174', '5175', '5176', '5177', '5178', '5179', '5180', '5181', '5182', 
    '5183', '5184', '5185', '5186', '5187', '5188', '5189', '5190', '5191', '5192', '5193', '5194', '5195', '5196', '5197', 
    '5198', '5199', '5200', '5600', '5601', '5602', '5603', '5604', '5605', '5606', '5607', '5608', '5609', '5610', '5611', 
    '5612', '5613', '5614', '5615', '5616', '5617', '5618', '5619', '5620', '5621', '5622', '5624', '5625', '5626', '5627', 
    '5628', '5629', '5630', '5631', '5632', '5633', '5634', '5635', '5636', '5637', '5638', '5639', '5640', '5641', '5642', 
    '5643', '5644', '5645', '5646', '5647', '5648', '5649', '5650', '5651', '5652', '5653', '5654', '5655', '5656', '5657', 
    '5658', '5659', '5660', '5661', '5662', '5663', '5664', '5665', '5666', '5667', '5668', '5669', '5670', '5671', '5672', 
    '5673', '5674', '5675', '5676', '5677', '5678', '5679', '5680', '5681', '5682', '5683', '5684', '5685', '5686', '5687', 
    '5688', '5689', '5690', '5691', '5692', '5693', '5694', '5695', '5696', '6001', '6002', '6003', '6004', '6005', '6009', 
    '6010', '6011', '6012', '6013', '6014', '6015', '6016', '6017', '6018', '100155', '100156', '100206', '100207'
]
SKU_ID_REGISTRATION = ['4001', '4017']

# Function to create engine connection to Athena
def create_engine_connection():
    connection_string = f"awsathena+rest://:@athena.{ATHENA_REGION}.amazonaws.com:443/labsprodeventsdatabase-x806vjuzpbrd?s3_staging_dir={S3_STAGING_DIR}"
    engine = create_engine(connection_string)
    return engine

# Function to fetch data for a specific date from Athena
def fetch_data_for_date(engine, date):
    year, month, day = date.strftime("%Y"), date.strftime("%m"), date.strftime("%d")
    query = f"""
    SELECT * FROM processed_events
    WHERE application_id='{APPLICATION_ID}'
    AND year='{year}' AND month='{month}' AND day='{day}'
    AND env = '{ENV}' AND user_type = '{USER_TYPE}'
    AND event_name IN ('ProfileCreated', 'OrderSubmitted')
    """
    df = pd.read_sql(query, engine)
    df_clean = clean_eReg_dataframe(df)
    return df_clean

# Function to generate a range of dates
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

# Function to extract SKU IDs from event data
def get_sku_id_list(df):
    """
    Extracts SKU IDs from order data, including user ID, order ID, 
    order submission time, and SKU ID list for each order.
    
    Parameters:
    df (DataFrame): The input DataFrame containing event data.
    
    Returns:
    DataFrame: A DataFrame containing user ID, order ID, order submission time, 
               and SKU ID list for each order.
    """
    # Define a list to hold our processed rows
    rows = []
 
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Load the JSON content from 'event_data' column
        event_data = json.loads(row['event_data'])
        # Extract necessary data
        user_id = row['user_id']
        order_id = event_data.get('data', {}).get('orderid')
        order_submit_time = row['event_timestamp']
        # Extract all SKU IDs from order items
        sku_ids = [item.get('skuid') for item in event_data.get('data', {}).get('orderitems', [])]
        # Append the extracted data as a dictionary to the rows list
        rows.append({
            'user_id': user_id,
            'order_id': order_id,
            'order_submit_time': order_submit_time,
            'sku_id_list': sku_ids
        })
    # Create a DataFrame from the rows list
    df = pd.DataFrame(rows)
    return df

# Function to update order DataFrame with specific SKU filters
def update_order_df(df, non_prep_SKUs, sku_id_registration):
    """
    Updates the order DataFrame by filtering out non-preparatory SKUs, 
    sorting orders within each user by submission time, and keeping only 
    the first order for each user. It also adds profile creation time and 
    time difference in seconds to the DataFrame.
    
    Parameters:
    df (DataFrame): The input DataFrame containing event data.
    non_prep_skus (list): List of non-preparatory SKU IDs.
    sku_id_registration (list): List of SKU IDs for registration.
    
    Returns:
    DataFrame: The updated DataFrame with filtered and sorted orders.
    """
    
    
    # Convert SKU lists to integers, filtering out non-integer strings
    non_prep_SKUs_int = [int(sku) for sku in non_prep_SKUs if sku.isdigit()]
    sku_id_registration_int = [int(sku) for sku in sku_id_registration if sku.isdigit()]
    # Convert lists to sets for operations
    non_prep_SKUs_set = set(non_prep_SKUs_int)
    sku_id_registration_set = set(sku_id_registration_int)
    # Compute the difference to find sku_id_fees
    sku_id_fees = non_prep_SKUs_set - sku_id_registration_set
 
    # Extract profile_created_time for each user
    profile_created_times = {}
    profile_created_rows = df[df['event_name'] == 'ProfileCreated']
    for index, row in profile_created_rows.iterrows():
        user_id = row['user_id']
        profile_created_time = row['event_timestamp']
        profile_created_times[user_id] = profile_created_time
 
    # Log missing user_ids for debugging
    missing_user_ids = []
 
    # List to collect rows for the new DataFrame
    updated_rows = []
    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Process only rows where event_name is OrderSubmitted
        if row['event_name'] != 'OrderSubmitted':
            continue
        # Load the JSON content from 'event_data' column
        event_data = json.loads(row['event_data'])
        # Extract necessary data
        user_id = row['user_id']
        order_id = event_data.get('data', {}).get('orderid')
        order_submit_time = row['event_timestamp']
        sku_ids = [int(item.get('skuid')) for item in event_data.get('data', {}).get('orderitems', []) if item.get('skuid', '')]
        # Filter out the SKUs that are only fee-related, leaving other types of SKUs
        filtered_sku_ids = [sku for sku in sku_ids if sku not in sku_id_fees]
        # Skip rows with empty SKU ID lists after filtering
        if not filtered_sku_ids:
            continue
        # Check if any SKU ID is a registration SKU
        registration_marker = 1 if any(sku in sku_id_registration_set for sku in filtered_sku_ids) else 0
        # Get profile_created_time for the user
        profile_created_time = profile_created_times.get(user_id)
        if profile_created_time is None:
            missing_user_ids.append(user_id)
        # Calculate time_diff_sec if profile_created_time is available
        time_diff_sec = None
        if profile_created_time is not None:
            time_diff_sec = (order_submit_time - profile_created_time) / 1e6  # Convert from microseconds to seconds
        # Append the processed data
        updated_rows.append({
            'user_id': user_id,
            'order_id': order_id,
            'order_submit_time': order_submit_time,
            'sku_id_list': filtered_sku_ids,
            'registration_marker': registration_marker,
            'profile_created_time': profile_created_time,
            'time_diff_sec': time_diff_sec
        })
    
    # Create a DataFrame from the updated rows
    updated_df = pd.DataFrame(updated_rows)
    # Filter to keep only rows where registration_marker is 0
    updated_df = updated_df[updated_df['registration_marker'] == 0]
    # Sort the DataFrame first by 'user_id', then by 'order_submit_time'
    updated_df.sort_values(by=['user_id', 'order_submit_time'], ascending=[True, True], inplace=True)
    # Keep only the first order for each user
    updated_df = updated_df.drop_duplicates(subset='user_id', keep='first')
    return updated_df

# Main script execution
if __name__ == "__main__":
    engine = create_engine_connection()
    data_frames = []
    
    for single_date in daterange(START_DATE, EXTENDED_END_DATE):
        print(f"Fetching data for {single_date.strftime('%Y-%m-%d')}...")
        df_clean = fetch_data_for_date(engine, single_date)
        data_frames.append(df_clean)
    
    full_data_frame = pd.concat(data_frames, ignore_index=True)
    df_updated = update_order_df(full_data_frame, NON_PREP_SKUS, SKU_ID_REGISTRATION)
    
    # Save the updated DataFrame as a parquet file
    df_updated.to_parquet('updated_orders.parquet')
    
    # Save the updated DataFrame as a CSV file
    df_updated.to_csv('updated_orders.csv', index=False)

    # Optionally display the DataFrame
    import ace_tools as tools; tools.display_dataframe_to_user(name="Updated Orders DataFrame", dataframe=df_updated)
